In [ ]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 235kB 2.9MB/s 
     |████████████████████████████████| 245kB 13.4MB/s 
     |████████████████████████████████| 112kB 13.2MB/s 
     |████████████████████████████████| 2.3MB 2.8MB/s 
     |████████████████████████████████| 3.3MB 19.9MB/s 
     |████████████████████████████████| 901kB 42.5MB/s 
  Found existing installation: huggingface-hub 0.0.9
    Uninstalling huggingface-hub-0.0.9:
      Successfully uninstalled huggingface-hub-0.0.9


In [ ]:
import transformers
import datasets

In [ ]:
import torch
from datasets import load_dataset

class NLITRReader(torch.utils.data.Dataset):
  def __init__(self, dataset_name, split_name, max_example_num=-1):
    self.dataset = load_dataset('nli_tr', dataset_name)
    self.split_name = split_name
    self.max_example_num = max_example_num

  def read(self):
      count = 0
      for example in self.dataset[self.split_name]:
          if example['label'] == -1: # skip examples having no gold value.
              continue
          count += 1
          if self.max_example_num > 0 and count >= self.max_example_num:
             break
          yield example

In [ ]:
import torch
class NLITRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
snli_tr_dataset = load_dataset('nli_tr', 'snli_tr')

Dataset nli_tr downloaded and prepared to /root/.cache/huggingface/datasets/nli_tr/snli_tr/1.0.0/c2ddd0c0a70caddac6a81c2dae5ca7939f00060d517d08f1983927818dba6521. Subsequent calls will reuse this data.


In [ ]:
print(snli_tr_dataset.shape)

{'train': (550152, 4), 'validation': (10000, 4), 'test': (10000, 4)}


In [127]:
train_data = snli_tr_dataset['train']
valid_data = snli_tr_dataset['validation']
test_data = snli_tr_dataset['test']

In [128]:
import pandas as pd
train_data_df = pd.DataFrame(train_data)

In [130]:
len(train_data_df)

550152

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [150]:
train_data_tokenized = pd.DataFrame(index = train_data_df.index,columns = ['hypothesis','index', 'label', 'premise'])
len(train_data_tokenized)

550152

In [151]:
for i in range(len(train_data_df)):
  print(word_tokenize(train_data[i]['hypothesis']))
  train_data_tokenized.iloc[i][0] = word_tokenize(train_data_df.iloc[i][0])
  print(train_data[i]['idx'])
  train_data_tokenized.iloc[i][1] = train_data_df.iloc[i][1]
  print(train_data[i]['label'])
  train_data_tokenized.iloc[i][2] = train_data_df.iloc[i][2]
  print(word_tokenize(train_data[i]['premise']))
  train_data_tokenized.iloc[i][3] = word_tokenize(train_data_df.iloc[i][3])


Streaming output truncated to the last 5000 lines.
['Gömlek', 'menekşesi', '.']
548902
1
['Mavi', 'gömlekli', 'bir', 'adam', 'elinde', 'bir', 'bardak', 'tutuyor', '.']
['Adam', 'bir', 'şey', 'tutuyor', '.']
548903
0
['Mavi', 'gömlekli', 'bir', 'adam', 'elinde', 'bir', 'bardak', 'tutuyor', '.']
['Seyirciler', 'dışarıda', '.']
548904
0
['Kayalık', ',', 'çimenli', 'bir', 'plajda', 'seyirciler', 'oturur', 've', 'gölde', 'yaşanan', 'olayları', 'izlerler', '.']
['Seyirciler', 'beyzbol', 'maçından', 'eve', 'dönüyor', '.']
548905
2
['Kayalık', ',', 'çimenli', 'bir', 'plajda', 'seyirciler', 'oturur', 've', 'gölde', 'yaşanan', 'olayları', 'izlerler', '.']
['Seyirciler', 'çok', 'ayrıntılı', 'bir', 'teklif', 'izliyorlar', '.']
548906
1
['Kayalık', ',', 'çimenli', 'bir', 'plajda', 'seyirciler', 'oturur', 've', 'gölde', 'yaşanan', 'olayları', 'izlerler', '.']
['Bisikletçi', 'gömlek', 'giymiyor', '.']
548907
2
['Kasklı', 've', 'sarı', 'jarseli', 'bir', 'bisikletçi', '.']
['Bisikletçi', 'bir', 'yarışt

In [152]:
train_data_tokenized

,hypothesis,index,label,premise
0,"[Bir, kişi, atını, yarışma, için, eğitiyor, .]",0,1,"[Attaki, bir, kişi, ,, bozuk, bir, uçağın, üze..."
1,"[Bir, kişi, bir, lokantada, omlet, sipariş, ed...",1,2,"[Attaki, bir, kişi, ,, bozuk, bir, uçağın, üze..."
2,"[Bir, kişi, açık, havada, ,, at, üzerinde, .]",2,0,"[Attaki, bir, kişi, ,, bozuk, bir, uçağın, üze..."
3,"[Ailelerine, gülümsüyorlar, .]",3,1,"[Fotoğraf, makinesinde, gülümseyen, ve, sallay..."
4,"[Burada, çocuklar, var, .]",4,0,"[Fotoğraf, makinesinde, gülümseyen, ve, sallay..."
...,...,...,...,...
550147,"[dört, çocuk'temiz, ayaklar'için, ödül, kazandı]",550147,2,"[Dört, kirli, ve, çıplak, ayaklı, çocuk, .]"
550148,"[Dört, evsiz, çocuğun, ayakkabısı, çalınmış, ,...",550148,1,"[Dört, kirli, ve, çıplak, ayaklı, çocuk, .]"
550149,"[Tulum, giyen, bir, adam, sörf, yarışmasında, ...",550149,1,"[Bir, adam, güzel, mavi, suda, bir, tulum, içi..."
550150,"[Takım, elbiseli, bir, adam, yönetim, kurulu, ...",550150,2,"[Bir, adam, güzel, mavi, suda, bir, tulum, içi..."


In [154]:
import os
#tokenized training data file
pd.DataFrame(train_data_tokenized).to_csv(os.getcwd()+'/trainData_nli_tr.csv') 


In [155]:
valid_data_df = pd.DataFrame(valid_data)

In [158]:
valid_data_tokenized = pd.DataFrame(index = valid_data_df.index,columns = ['hypothesis','index', 'label', 'premise'])
len(valid_data_tokenized)

10000

In [161]:
for i in range(len(valid_data_df)):
  print(word_tokenize(valid_data_df.iloc[i][0]))
  valid_data_tokenized.iloc[i][0] = word_tokenize(valid_data_df.iloc[i][0])
  print(valid_data_df.iloc[i][1])
  valid_data_tokenized.iloc[i][1] = valid_data_df.iloc[i][1]
  print(valid_data_df.iloc[i][2])
  valid_data_tokenized.iloc[i][2] = valid_data_df.iloc[i][2]
  print(word_tokenize(valid_data_df.iloc[i][3]))
  valid_data_tokenized.iloc[i][3] = word_tokenize(valid_data_df.iloc[i][3])
  

Streaming output truncated to the last 5000 lines.
['Bir', 'grup', 'arkadaş', 'birlikte', 'kahve', 'içiyor', '.']
8750
1
['İki', 'erkekle', 'oturan', 'bir', 'kadın', 'kucaklaşıyor', '.']
['Bir', 'grup', 'insan', 'birlikte', 'oturuyor', '.']
8751
0
['İki', 'erkekle', 'oturan', 'bir', 'kadın', 'kucaklaşıyor', '.']
['Toprak', 'pistinde', 'bir', 'yarış', '.']
8752
1
['Mavi', 'yarış', 'arabası', 'seyirci', 'fotoğraf', 'çekmek', 'gibi', 'yolda', 'toz', 'bulutu', 'yükseltir', '.']
['İnsanlar', 'araba', 'izliyor', '.']
8753
0
['Mavi', 'yarış', 'arabası', 'seyirci', 'fotoğraf', 'çekmek', 'gibi', 'yolda', 'toz', 'bulutu', 'yükseltir', '.']
['Feribot', 'teknesine', 'park', 'etmiş', 'bir', 'araba', '.']
8754
2
['Mavi', 'yarış', 'arabası', 'seyirci', 'fotoğraf', 'çekmek', 'gibi', 'yolda', 'toz', 'bulutu', 'yükseltir', '.']
['Bir', 'adam', 'kayıkla', 'nehirden', 'aşağı', 'iniyor', '.']
8755
1
['Sprey', 'kırmızı', 'kayık', 'adam', 'wielding', 'kürek', 'yüzünü', 've', 'gövdesini', 'gizler', '.']
['Bir

In [163]:
#tokenized validation  data file
pd.DataFrame(valid_data_tokenized).to_csv(os.getcwd()+'/validData_nli_tr.csv') 

In [165]:
test_data_df = pd.DataFrame(test_data)

In [166]:
test_data_tokenized = pd.DataFrame(index = test_data_df.index,columns = ['hypothesis','index', 'label', 'premise'])
len(test_data_tokenized)

10000

In [167]:
for i in range(len(test_data_df)):
  print(word_tokenize(test_data_df.iloc[i][0]))
  test_data_tokenized.iloc[i][0] = word_tokenize(test_data_df.iloc[i][0])
  print(test_data_df.iloc[i][1])
  test_data_tokenized.iloc[i][1] = test_data_df.iloc[i][1]
  print(test_data_df.iloc[i][2])
  test_data_tokenized.iloc[i][2] = test_data_df.iloc[i][2]
  print(word_tokenize(test_data_df.iloc[i][3]))
  test_data_tokenized.iloc[i][3] = word_tokenize(test_data_df.iloc[i][3])
  

Streaming output truncated to the last 5000 lines.
['Kumaş', 'dökümlü', 'bir', 'odada', 'masanın', 'yanında', 'oturan', 'adam', '.']
8750
0
['Bir', 'adam', 'duvarlardan', 'aşağı', 'dökümlü', 'kumaşlarla', 'bir', 'odanın', 'içinde', 'dekore', 'edilmiş', 'bir', 'masanın', 'arkasında', 'oturur', '.']
['Bir', 'tarotta', 'kart', 'dükkanında', 'oturan', ',', 'avucunun', 'okunmasını', 'bekleyen', 'adam', '.']
8751
1
['Bir', 'adam', 'duvarlardan', 'aşağı', 'dökümlü', 'kumaşlarla', 'bir', 'odanın', 'içinde', 'dekore', 'edilmiş', 'bir', 'masanın', 'arkasında', 'oturur', '.']
['Kimsenin', 'dövmesi', 'yok', '.']
8752
2
['Kolunda', 'bir', 'melek', 'dövmesi', 'olan', 'sarışın', 'bir', 'kadın', 'sokakta', 'yürürken', ',', 'arka', 'planda', 'beton', 'bir', 'duvar', '.']
['Dövmeli', 'bir', 'kişi']
8753
0
['Kolunda', 'bir', 'melek', 'dövmesi', 'olan', 'sarışın', 'bir', 'kadın', 'sokakta', 'yürürken', ',', 'arka', 'planda', 'beton', 'bir', 'duvar', '.']
['Harika', 'bir', 'dövme', 'olan', 'bir', 'kişi']
8

In [168]:
#tokenized test data file
pd.DataFrame(test_data_tokenized).to_csv(os.getcwd()+'/testData_nli_tr.csv') 

In [169]:
# df = pd.read_csv('trainData_nli_tr.csv')
# df

,Unnamed: 0,hypothesis,index,label,premise
0,0,"['Bir', 'kişi', 'atını', 'yarışma', 'için', 'e...",0,1,"['Attaki', 'bir', 'kişi', ',', 'bozuk', 'bir',..."
1,1,"['Bir', 'kişi', 'bir', 'lokantada', 'omlet', '...",1,2,"['Attaki', 'bir', 'kişi', ',', 'bozuk', 'bir',..."
2,2,"['Bir', 'kişi', 'açık', 'havada', ',', 'at', '...",2,0,"['Attaki', 'bir', 'kişi', ',', 'bozuk', 'bir',..."
3,3,"['Ailelerine', 'gülümsüyorlar', '.']",3,1,"['Fotoğraf', 'makinesinde', 'gülümseyen', 've'..."
4,4,"['Burada', 'çocuklar', 'var', '.']",4,0,"['Fotoğraf', 'makinesinde', 'gülümseyen', 've'..."
...,...,...,...,...,...
550147,550147,"['dört', ""çocuk'temiz"", ""ayaklar'için"", 'ödül'...",550147,2,"['Dört', 'kirli', 've', 'çıplak', 'ayaklı', 'ç..."
550148,550148,"['Dört', 'evsiz', 'çocuğun', 'ayakkabısı', 'ça...",550148,1,"['Dört', 'kirli', 've', 'çıplak', 'ayaklı', 'ç..."
550149,550149,"['Tulum', 'giyen', 'bir', 'adam', 'sörf', 'yar...",550149,1,"['Bir', 'adam', 'güzel', 'mavi', 'suda', 'bir'..."
550150,550150,"['Takım', 'elbiseli', 'bir', 'adam', 'yönetim'...",550150,2,"['Bir', 'adam', 'güzel', 'mavi', 'suda', 'bir'..."
